#first we integerate the data and label using panda an os libraries

In [23]:
import os
import pandas as pd

dataset_path = "C:\\Users\\mohamed\\OneDrive\\Desktop\\Husky_Wolf project\\dataset"
categories = ["huskies", "wolves"]
data = []
for category in categories:
    folder_path = os.path.join(dataset_path, category) 
    label = categories.index(category)
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        data.append([image_path, label])  # Store path and label
images = pd.DataFrame(data, columns=["image_path", "label"])
print(images.head)


<bound method NDFrame.head of                                            image_path  label
0   C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      0
1   C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      0
2   C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      0
3   C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      0
4   C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      0
..                                                ...    ...
95  C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      1
96  C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      1
97  C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      1
98  C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      1
99  C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      1

[100 rows x 2 columns]>


In [24]:
from sklearn.model_selection import train_test_split

train_im, test_im = train_test_split(images, test_size=0.2, random_state=42, stratify=images["label"])
print(train_im.head())


                                           image_path  label
15  C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      0
52  C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      1
60  C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      1
66  C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      1
68  C:\Users\mohamed\OneDrive\Desktop\Husky_Wolf p...      1


now we need to preprocess the dataset

In [ ]:
import torch  
from torch.utils.data import Dataset, DataLoader  
from torchvision import transforms  
import cv2 
import albumentations as A  
from albumentations.pytorch import ToTensorV2  
# Define augmentation transformations for training data
train_transforms = A.Compose([
    A.Resize(128, 128),  
    A.HorizontalFlip(p=0.5),  
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=20, p=0.5),  
    A.RandomResizedCrop(size=(128, 128), scale=(0.8, 1.0), ratio=(0.75, 1.33), p=0.5),  
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), 
    ToTensorV2(), 
    ])
test_transforms = A.Compose([
    A.Resize(128, 128), 
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  
    ToTensorV2(), 
])



creating dataset class

In [30]:
class HuskyWolfDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe  
        self.transform = transform 
    def __len__(self):
        return len(self.dataframe)  
    def __getitem__(self, index):
        image_path = self.dataframe.iloc[index]["image_path"]  
        label = self.dataframe.iloc[index]["label"]  

        # Load image using OpenCV
        image = cv2.imread(image_path)  # Read image (BGR format)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Apply transformations (if any)
        if self.transform:
            augmented = self.transform(image=image)  # Apply transformations
            image = augmented["image"]

        return image, torch.tensor(label, dtype=torch.long)  # Return image & label as tensors


making instance of it

In [29]:
train_dataset = HuskyWolfDataset(train_im, transform=train_transforms)  
test_dataset = HuskyWolfDataset(test_im, transform=test_transforms)  


# Create DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)
